# Things to try

* [a Treemap](https://vega.github.io/vega/examples/treemap/)

In [1]:
from sqlalchemy import create_engine
engine = create_engine(f"sqlite:///birdnest.db")
import pandas as pd
import altair as alt
from spotclient import Client
import models

db = models.Database()
session = models.get_session()


In [103]:
session = models.get_session()
playlist = session.query(models.Playlist).first()

pdf = pd.DataFrame(playlist.to_json(True))

In [104]:
from functools import reduce
pljson = map(lambda x: x.to_json(True),session.query(models.Playlist))
cumpl = [] # seems like I should be able to do this with reduce but it's erroring
for pl in pljson:
    cumpl.extend(pl)
    

In [113]:
[x for x in cumpl if x['name'] == 'Ghost Riders In The Sky']

[{'track_id': '7lgUKKPtebsxG500ZNNjiT',
  'name': 'Ghost Riders In The Sky',
  'duration_ms': 276266,
  'explicit': False,
  'popularity': 4,
  'album': 'Cowboy Rumba',
  'artists': 'Ned Sublette',
  'key': 5,
  'key_str': 'F',
  'mode': 0,
  'mode_str': 'major',
  'tempo': 138.048,
  'time_signature': 4,
  'acousticness': 0.332,
  'danceability': 0.768,
  'energy': 0.883,
  'instrumentalness': 0.0627,
  'liveness': 0.321,
  'loudness': -5.136,
  'speechiness': 0.0445,
  'valence': 0.914,
  'date': '2020-05-28',
  'year': 2020,
  'month': 5,
  'day': 28,
  'start_time_ms': 10259290},
 {'track_id': '3BgDkANwnqlEDUzLevbdkx',
  'name': 'Ghost Riders In The Sky',
  'duration_ms': 215266,
  'explicit': False,
  'popularity': 1,
  'album': 'The Singing Cowboy: Chapter One',
  'artists': 'Gene Autry',
  'key': 0,
  'key_str': 'C',
  'mode': 0,
  'mode_str': 'major',
  'tempo': 121.406,
  'time_signature': 4,
  'acousticness': 0.977,
  'danceability': 0.421,
  'energy': 0.178,
  'instrumentaln

In [105]:
len(cumpl)

1898

In [114]:
alt.Chart(alt.Data(values=cumpl[:1000])).mark_bar().encode(
    alt.X('name:N', title='', sort='y',axis=alt.Axis(labels=False)),
    alt.Y('acousticness:Q', title='acousticness'),
    tooltip=alt.Tooltip(['name:N','artists:N','acousticness:Q','time_signature:Q', 'track_id:N']),
    color=alt.condition(alt.datum.acousticness > .9, alt.value('red'), alt.value('lightgrey'))
).properties(width=500)

alt.Chart(...)

In [115]:
c2 = list(map(lambda d: dict((k,v) for k,v in d.items() if k in ['name','artists','acousticness','time_signature', 'track_id']), cumpl))

In [116]:
alt.Chart(alt.Data(values=c2[:1000])).mark_bar().encode(
    alt.X('name:N', title='', sort='y',axis=alt.Axis(labels=False)),
    alt.Y('acousticness:Q', title='acousticness'),
    tooltip=alt.Tooltip(['name:N','artists:N','acousticness:Q','time_signature:Q', 'track_id:N']),
    color=alt.condition(alt.datum.acousticness > .9, alt.value('red'), alt.value('lightgrey'))
).properties(width=500)

alt.Chart(...)

In [35]:
alt.Chart(alt.Data(values=playlist.to_json(True))).mark_bar().encode(
    alt.X('energy:Q',bin=True, title='Energy (histogram)'),
    alt.Y(aggregate='count',type='quantitative', title='Number of tracks'))
)

alt.Chart(...)

In [32]:
def playlist_as_df(cols=None):
    if cols is None:
        af_clause = 'af.*'
    elif isinstance(cols,str):
        af_clause = cols
    else:
        clause_cols = []
        for c in cols:
            if not c.startswith('af.'):
                c = f"af.{c}"
            clause_cols.append(c)
        af_clause = ','.join(clause_cols)
    
    SQL = f"""
    SELECT
        p.date, 
        p.name playlist_name, p.playlist_id,
        t.name,
        GROUP_CONCAT(a.name,';') artist,
        t.duration_ms,
        {af_clause}
    FROM
        playlist p,
        playlist_track pt,
        track t,
        track_artist ta,
        artist a,
        audio_features af
    where t.track_id = ta.track_id 
         and ta.artist_id = a.artist_id
         and p.playlist_id = pt.playlist_id
         and pt.track_id = t.track_id
         and af.track_id = t.track_id
    group by t.name 
"""

    engine = create_engine(f"sqlite:///birdnest.db")
    return pd.read_sql(SQL, engine)


In [44]:
sql = '''
select artist.name, count(*) count
from artist, track_artist, playlist_track
where artist.artist_id = track_artist.artist_id
and track_artist.track_id = playlist_track.track_id
group by artist.name
'''
artists_df = pd.read_sql(sql,engine)
alt.Chart
artists_df.sort_values('count',ascending=False).head()

,name,count
1664,The Upsetters,8
170,Björk,8
636,Herbie Hancock,8
1782,Willie Nelson,8
625,Hassan Hakmoun,6


In [53]:
alt.Chart(artists_df[artists_df['count'] >= 5].sort_values('count',ascending=False)).mark_bar().encode(
    x=alt.X('count',title='times played'),
    y=alt.Y('name',sort='-x', title='Artist')
)

alt.Chart(...)

In [71]:
alt.Chart(artists_df.groupby('count').count().reset_index().rename(columns={
    'name': '# Artists played X times'
})).mark_bar().encode(
        x=alt.X('# Artists played X times'),
        y=alt.Y('count:N',sort='-y'),
        tooltip=['count', '# Artists played X times']
    )

alt.Chart(...)

## Genre frequency

Genres attach to artists, not tracks, tracks can have multiple artists... the meaning of this is suspect but was curious

In [82]:
# 
sql = '''
select track.name track, genre.name genre
from track, track_artist, artist_genre, genre
where track.track_id = track_artist.track_id 
and track_artist.artist_id = artist_genre.artist_id
and artist_genre.genre_id = genre.genre_id
'''
df = pd.read_sql(sql,engine).groupby('genre').count().rename(columns={'track': 'tracks'}).reset_index()
alt.Chart(df.sort_values('tracks',ascending=False).head(50)).mark_bar().encode(
    x='tracks',
    y=alt.Y('genre:N',sort='-x')
)

alt.Chart(...)

In [39]:
def playlist_stats(df):
    """
        Given a dataframe, produce a summarization in the same shape as AudioFeature,
        but aggregated. For now, aggregated as the sum of (each feature * track length)
        but that could become switchable.

        'key', 'mode': probably the mode for each set, and another for the two as a unit?
        'tempo': average
        'time_signature', maybe mode or mode
       'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence'
    """
    for 
    df = df
df['gross_valence'] = df['valence'] * df['duration_ms']
df.groupby('playlist_name').sum()[['gross_valence']].sort_values('gross_valence',ascending=False)


,gross_valence
playlist_name,
JQBX :: Conference of the Birds 2020-05-28,1.037720e+07
JQBX :: Conference of the Birds 2020-12-30,7.494227e+06
JQBX :: Conference of the Birds 2020-11-26,7.371483e+06
JQBX :: Conference of the Birds 2021-03-04,7.078833e+06
JQBX :: Conference of the Birds 2020-05-14,7.031170e+06
JQBX :: Conference of the Birds 2021-01-07,6.920893e+06
JQBX :: Conference of the Birds 2020-09-24,6.818546e+06
💫 JQBX :: Conference of the Birds 2020-04-16,6.804963e+06
JQBX :: Conference of the Birds 2020-12-03,6.630199e+06


In [12]:
print(duration)

0:01:00


In [167]:
previewless = session.query(models.Track).filter(models.Track.preview_url == None).all()
t_info = Client().tracks([t.spotify_id for t in previewless[:10]])

In [171]:
"{:,}".format(10000)

'10,000'

In [108]:
alt.Chart(alt.Data(values=list(artist_count))).mark_bar().encode(
    x=alt.X('count:Q',title='times played'),
    y=alt.Y('name:N',sort='-x', title='Artist')
).transform_filter(datum.count >= )

alt.Chart(...)

In [ ]:
q = session.query(models.Track)
q
#pd.read_sqjoin.jo.statement,session.bind)